In [2]:
from bokeh.plotting import figure, output_file, show
from bokeh.models.widgets import Dropdown
from bokeh.io import output_file, show
#from bokeh.models.widgets.layouts import VBox,HBox
import pandas as pd
from bokeh.plotting import curdoc
from bokeh.models import Select
from bokeh.layouts import row, widgetbox
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.charts import Scatter
from bokeh.models import LinearAxis
from bokeh.models.widgets import Panel, Tabs
from bokeh.charts import Histogram
from bokeh.charts import defaults
from bokeh.models import HoverTool, BoxSelectTool, ResetTool,PanTool,BoxZoomTool
from bokeh.charts import HeatMap
from bokeh.layouts import layout
from bokeh.layouts import row, widgetbox
from bokeh.models import CheckboxGroup, CustomJS
from bokeh.charts import Bar
from collections import OrderedDict
from bokeh.io import push_notebook, show, output_notebook
output_notebook()
import numpy as np
from bokeh.io import curdoc

curdoc().clear()

from bokeh.document import Document
from bokeh.embed import file_html
from bokeh.layouts import gridplot
from bokeh.models.glyphs import Circle
from bokeh.models import (BasicTicker, ColumnDataSource, Grid, LinearAxis,
                         DataRange1d, PanTool, Plot, WheelZoomTool)
from bokeh.resources import INLINE
from bokeh.sampledata.iris import flowers
from bokeh.util.browser import view

df=pd.read_csv("cellPhone.csv")
#population dataframe
dfP=pd.read_csv("population.csv")
headers=df.columns
headers=[str(r) for r in headers]


#by country, by year, total
#check recent 10 years, 2002 to 2011
yearList=[]
for i in range(2002,2012):
	yearList.append(i)


USAList=[]
USAPopulation=[]
USARate=[]
ChinaList=[]
ChinaPopulation=[]
ChinaRate=[]
HaitiList=[]
HaitiPopulation=[]
HaitiRate=[]



def changeInt(num):
	return int(num.replace(',',''))


def calculateRate(rawStringOne,rawStringTwo):
	previous=changeInt(rawStringOne)
	current=changeInt(rawStringTwo)
	rate=float(current-previous)/(previous)*100
	return rate

#print "cell is: ",df['2002'][df.loc[df['Mobile cellular subscriptions (per 100 people)']=='United States']]
#for i in yearList:
	#USAList.append(df[i]['Mobile cellular subscriptions (per 100 people)'])
for i in yearList:
	USAList.append(df.loc[df['Mobile cellular subscriptions (per 100 people)']=='United States',str(i)].iloc[0])
	USAPopulation.append(dfP.loc[dfP['Total population']=='United States',str(i)].iloc[0])
	a=i-1
	
	#rateUSA=(dfP.loc[dfP['Total population']=='United States',str(i)].iloc[0]-dfP.loc[dfP['Total population']=='United States',str(a)].iloc[0])/dfP.loc[dfP['Total population']=='United States',str(a)].iloc[0]
	rateUSA=calculateRate((dfP.loc[dfP['Total population']=='United States',str(a)].iloc[0]),dfP.loc[dfP['Total population']=='United States',str(i)].iloc[0])
	
	USARate.append(rateUSA)



	ChinaList.append(df.loc[df['Mobile cellular subscriptions (per 100 people)']=='China',str(i)].iloc[0])
	ChinaPopulation.append(dfP.loc[dfP['Total population']=='China',str(i)].iloc[0])
	#rateChina=(dfP.loc[dfP['Total population']=='China',str(i)].iloc[0]-dfP.loc[dfP['Total population']=='China',str(a)].iloc[0])/dfP.loc[dfP['Total population']=='China',str(a)].iloc[0]
	rateChina=calculateRate(dfP.loc[dfP['Total population']=='China',str(a)].iloc[0],dfP.loc[dfP['Total population']=='China',str(i)].iloc[0])
	
	ChinaRate.append(rateChina)

	HaitiList.append(df.loc[df['Mobile cellular subscriptions (per 100 people)']=='Haiti',str(i)].iloc[0])
	HaitiPopulation.append(dfP.loc[dfP['Total population']=='Haiti',str(i)].iloc[0])
	#rateHaiti=(dfP.loc[dfP['Total population']=='Haiti',str(i)].iloc[0]-dfP.loc[dfP['Total population']=='Haiti',str(a)].iloc[0])/dfP.loc[dfP['Total population']=='Haiti',str(a)].iloc[0]
	rateHaiti=calculateRate(dfP.loc[dfP['Total population']=='Haiti',str(a)].iloc[0],dfP.loc[dfP['Total population']=='Haiti',str(i)].iloc[0])
	HaitiRate.append(rateHaiti)
	


TOOLS=[BoxSelectTool(), HoverTool(),ResetTool(),PanTool(),BoxZoomTool()]
TOOLS2=[BoxSelectTool(), HoverTool(),ResetTool(),PanTool(),BoxZoomTool()]
p = figure(plot_width=650, plot_height=500,title="growth of mobile subscirbes in 10 years",tools=TOOLS,x_axis_label='year',y_axis_label='mobile phone subscribes in 100 persons')
lChina = p.line(yearList,ChinaList, legend="China", line_color="red",line_width=2)
lUSA = p.line(yearList,USAList, legend="US", line_color="blue",line_width=2)
lHaiti=p.line(yearList,HaitiList, legend="Haiti", line_color="green",line_width=2)
p2 = figure(plot_width=650, plot_height=500,title="rate of population growth",tools=TOOLS2,x_range=p.x_range,x_axis_label='year',y_axis_label='growth rate')
#x_range=p.x_range

lChinaRate = p2.line(yearList,ChinaRate, legend="China Rate", line_color="red",line_width=2)
lUSARate = p2.line(yearList,USARate, legend="US Rate", line_color="blue",line_width=2)
lHaitiRate=p2.line(yearList,HaitiRate, legend="Haiti", line_color="green",line_width=2)



code='''\
lChina.visible = 0 in checkbox.active;
lUSA.visible = 1 in checkbox.active;
lHaiti.visible = 2 in checkbox.active;

'''

checkbox = CheckboxGroup(labels=['China','USA','Haiti'],
                         active=[0,1,2], width=100)


checkbox.callback = CustomJS.from_coffeescript(code=code, args=dict(lChina=lChina,lUSA=lUSA,lHaiti=lHaiti,checkbox=checkbox),)
stackYearList=[]

for j in range(0,3):
  for i in range(2002,2012):
    stackYearList.append(i)

stackedCountryList=[]
countryList=['USA','China','Haiti']

for c in countryList:
  for i in range(2002,2012):
    stackedCountryList.append(c)


data=[]
data=USAList+ChinaList+HaitiList
all_data={'year':stackYearList,'country':stackedCountryList,'data':data}
dfBar=pd.DataFrame(all_data)
bar=Bar(dfBar,label='year',values='data',title='stacked bar for 3 countries',stack='country',legend='top_right')   


layout=layout([bar],[row(p,p2)],[checkbox])
show(layout)




Loading BokehJS ...

[2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011]
